<a href="https://colab.research.google.com/github/bhuvan0808/Machine_learning/blob/main/Customer_Segmentation_Kmeans_and_HC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES
# TO THE CORRECT LOCATION (/kaggle/input) IN YOUR NOTEBOOK,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

import os
import sys
from tempfile import NamedTemporaryFile
from urllib.request import urlopen
from urllib.parse import unquote, urlparse
from urllib.error import HTTPError
from zipfile import ZipFile
import tarfile
import shutil

CHUNK_SIZE = 40960
DATA_SOURCE_MAPPING = 'customer-segmentation-dataset:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F1835880%2F2996440%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240229%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240229T004610Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3Dac1e4f7e2679823c1a6df1fac8183ff14c86bb31e323bda9f0c9e14bf40553b2df692d7c2d5207d08a9f43e0cc2ad4cd12fae3297660763589e71083b2683f35c7b7209de52a549d3bdbdcd9791c296971839e577ed8cd74130ed047889d55124f5e51f2ce8dc6ff1ab2abf904358bf77a636eca2311628ffba3adcf1a5b50d50b905044411e479401e03bc8b7da278274dd961a9b7583baa19e2b7ee95e2fbaa3312c59ed926285fe1a6e01b2b295f8148b62ace314b61438d33d3e01ac25c33adea117fe4b5c8fcc35f43d2fb58158077eb3645e5a7d713d667cb82ee4d2a83ef5849d9ed48c62359b33b297f38bada05ccb5f8bebab0c6ff389bb47f8bca8'

KAGGLE_INPUT_PATH='/kaggle/input'
KAGGLE_WORKING_PATH='/kaggle/working'
KAGGLE_SYMLINK='kaggle'

!umount /kaggle/input/ 2> /dev/null
shutil.rmtree('/kaggle/input', ignore_errors=True)
os.makedirs(KAGGLE_INPUT_PATH, 0o777, exist_ok=True)
os.makedirs(KAGGLE_WORKING_PATH, 0o777, exist_ok=True)

try:
  os.symlink(KAGGLE_INPUT_PATH, os.path.join("..", 'input'), target_is_directory=True)
except FileExistsError:
  pass
try:
  os.symlink(KAGGLE_WORKING_PATH, os.path.join("..", 'working'), target_is_directory=True)
except FileExistsError:
  pass

for data_source_mapping in DATA_SOURCE_MAPPING.split(','):
    directory, download_url_encoded = data_source_mapping.split(':')
    download_url = unquote(download_url_encoded)
    filename = urlparse(download_url).path
    destination_path = os.path.join(KAGGLE_INPUT_PATH, directory)
    try:
        with urlopen(download_url) as fileres, NamedTemporaryFile() as tfile:
            total_length = fileres.headers['content-length']
            print(f'Downloading {directory}, {total_length} bytes compressed')
            dl = 0
            data = fileres.read(CHUNK_SIZE)
            while len(data) > 0:
                dl += len(data)
                tfile.write(data)
                done = int(50 * dl / int(total_length))
                sys.stdout.write(f"\r[{'=' * done}{' ' * (50-done)}] {dl} bytes downloaded")
                sys.stdout.flush()
                data = fileres.read(CHUNK_SIZE)
            if filename.endswith('.zip'):
              with ZipFile(tfile) as zfile:
                zfile.extractall(destination_path)
            else:
              with tarfile.open(tfile.name) as tarfile:
                tarfile.extractall(destination_path)
            print(f'\nDownloaded and uncompressed: {directory}')
    except HTTPError as e:
        print(f'Failed to load (likely expired) {download_url} to path {destination_path}')
        continue
    except OSError as e:
        print(f'Failed to load {download_url} to path {destination_path}')
        continue

print('Data source import complete.')


<center><img src="https://raw.githubusercontent.com/Masterx-AI/Project_Customer_Segmentation_/main/customer_segmentation.png" style="width: 600px;"/>

In [ ]:
# import required libraries for dataframe and visualization

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns
import datetime as dt
import plotly.express as px


# import required libraries for clustering
import sklearn
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from scipy.cluster.hierarchy import linkage
from scipy.cluster.hierarchy import dendrogram
from scipy.cluster.hierarchy import cut_tree
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.cluster import KMeans

import warnings
warnings.filterwarnings('ignore')

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))


In [ ]:
retail = pd.read_excel('/kaggle/input/customer-segmentation-dataset/Online Retail.xlsx')
retail.head()

In [ ]:
retail.shape


The "Online Retail" dataset likely contains information about online retail transactions. Let's go through the meaning of each column in the dataset:

InvoiceNo: This column likely contains a unique identifier or code for each retail invoice or transaction. It helps in tracking and identifying individual sales transactions.

StockCode: This column likely contains a code or identifier for the specific product or item being sold in each transaction. It allows the association of products with their respective transactions.

Description: This column probably contains a textual description or name of the product or item being sold. It provides more detailed information about the product.

Quantity: This column likely represents the quantity of the product or item that was sold in each transaction. It indicates how many units of the product were purchased.

InvoiceDate: This column probably records the date and time when each retail invoice or transaction occurred. It provides a timestamp for when the sale took place.

UnitPrice: This column contains the price per unit of the product or item being sold. It represents the cost of one unit of the product.

CustomerID: This column likely contains a unique identifier or code for each customer who made a purchase. It allows tracking of customer-specific transactions.

Country: This column contains the name or code of the country where the customer who made the purchase is located. It provides geographic information about the customers.

This dataset appears to be a record of online retail transactions, including details about the products sold, quantities, prices, customer information, and transaction timestamps. It is commonly used for various types of retail analytics, such as customer segmentation, sales forecasting, and product performance analysis. The dataset can provide insights into customer behavior and sales patterns in the online retail business.

In [ ]:
retail.info()


In [ ]:
retail.describe()


# Data Cleaning


In [ ]:
# Calculating the Missing Values % contribution in DF

df_null = round(100*(retail.isnull().sum())/len(retail), 2)
df_null

In [ ]:
# Droping rows having missing values

retail = retail.dropna()
retail.shape

In [ ]:
# Changing the datatype of Customer Id as per Business understanding

retail['CustomerID'] = retail['CustomerID'].astype(str)

# Feature Engineering

In [ ]:
# New Attribute : Monetary

retail['Amount'] = retail['Quantity']*retail['UnitPrice']
retail.head()

In [ ]:
# This step groups the data in the retail DataFrame by the 'CustomerID' column
# to calculates the sum of the 'Amount' column for each customer.
rfm_m = retail.groupby('CustomerID')['Amount'].sum().reset_index()
rfm_m.head()


In [ ]:
# New Attribute : Frequency
# count the number of unique invoice numbers for each customer
rfm_f = retail.groupby('CustomerID')['InvoiceNo'].count().reset_index()
rfm_f.head()

In [ ]:
rfm_f.columns

In [ ]:
rfm_f.rename(columns={'InvoiceNo': 'Frequency'}, inplace=True)
rfm_f.head()

In [ ]:
# Merging the two dfs

merged_rfm = rfm_m.merge(rfm_f, on='CustomerID')
# another way
# rfm = pd.merge(rfm_m, rfm_f, on='CustomerID', how='inner')
merged_rfm.head()

In [ ]:
retail.info()

In [ ]:

# Convert to datetime to proper datatype

retail['InvoiceDate'] = pd.to_datetime(retail['InvoiceDate'],format='%d-%m-%Y %H:%M')
retail.head()

In [ ]:
# Compute the maximum date to know the last transaction date in our dataset

max_date = max(retail['InvoiceDate'])
max_date

In [ ]:
# calculate the time difference between the maximum date in the dataset and the transaction date for each record

retail['Diff'] = max_date - retail['InvoiceDate']
retail.head()

In [ ]:
# Compute last transaction date to get the recency of customers
# As each customer may have multiple invonces that has been purshsed
# So we need to create a indicator that tells us when he made the last transaction to find wather he is active or not
rfm_p = retail.groupby('CustomerID')['Diff'].min().reset_index()
rfm_p.head()

In [ ]:
# Extract number of days only

rfm_p['Diff'] = rfm_p['Diff'].dt.days
rfm_p.head()

In [ ]:
merged_rfm.head()

In [ ]:
# Merge tha dataframes to get the final RFM dataframe

merged_rfm = merged_rfm.merge(rfm_p, on='CustomerID')

merged_rfm.rename(columns={'Diff': 'Recency'}, inplace=True)
merged_rfm.head()


There are 2 types of outliers and we will treat outliers as it can skew our dataset

Statistical

Domain specific

In [ ]:

# Create a DataFrame containing the selected attributes
attributes = ['Amount', 'Frequency', 'Recency']
data = merged_rfm[attributes]

# Create a box plot with Plotly
fig = px.box(data, y=attributes, labels={'variable': 'Attributes', 'value': 'Range'},
             title="Outliers Variable Distribution")
fig.update_layout(
    xaxis=dict(title="Attributes"),
    yaxis=dict(title="Range"),
    showlegend=False,
    boxmode='group',  # Display box plots side by side
    width=800, height=600
)
fig.show()


In [ ]:

# Removing (statistical) outliers for Amount
Q1 = merged_rfm.Amount.quantile(0.05)
Q3 = merged_rfm.Amount.quantile(0.95)
IQR = Q3 - Q1
merged_rfm = merged_rfm[(merged_rfm.Amount >= Q1 - 1.5*IQR) & (merged_rfm.Amount <= Q3 + 1.5*IQR)]



# Removing (statistical) outliers for Recency
Q1 = merged_rfm.Recency.quantile(0.05)
Q3 = merged_rfm.Recency.quantile(0.95)
IQR = Q3 - Q1
merged_rfm = merged_rfm[(merged_rfm.Recency >= Q1 - 1.5*IQR) & (merged_rfm.Recency <= Q3 + 1.5*IQR)]

# Removing (statistical) outliers for Frequency
Q1 = merged_rfm.Frequency.quantile(0.05)
Q3 = merged_rfm.Frequency.quantile(0.95)
IQR = Q3 - Q1
merged_rfm = merged_rfm[(merged_rfm.Frequency >= Q1 - 1.5*IQR) & (merged_rfm.Frequency <= Q3 + 1.5*IQR)]

#### Purpose of Outlier Removal:

**Outliers can distort statistical analyses and models by introducing noise and skewing results.**

**Removing outliers helps in obtaining a more representative view of the dataset and its underlying patterns.**

**It can improve the accuracy and reliability of insights derived from the data.**

**However, removing outliers should be done carefully, and the choice of the 1.5*IQR range is a common heuristic.**

**It's essential to consider the context and business goals.**

**In summary, the provided code removes statistical outliers from the "Amount," "Recency," and "Frequency" attributes to ensure that subsequent analyses and modeling are based on a cleaner and more representative dataset.**

In [ ]:

attributes = ['Amount', 'Frequency', 'Recency']

fig = px.box(merged_rfm, y=attributes, title="Outliers Variable Distribution",
             labels={'variable': 'Attributes', 'value': 'Range'},
             boxmode='group', points='outliers')

fig.update_layout(
    xaxis=dict(title="Attributes", title_font=dict(size=14)),
    yaxis=dict(title="Range", title_font=dict(size=14)),
    showlegend=False,
    width=800,
    height=600
)

fig.show()


## Rescaling the Attributes
#### It is extremely important to rescale the variables so that they have a comparable scale.| There are two common ways of rescaling:

##### 1-Min-Max scaling
##### 2-Standardisation (mean-0, sigma-1)

Here, we will use Standardisation Scaling.

In [ ]:
# Rescaling the attributes

merged_rfm = merged_rfm[['Amount', 'Frequency', 'Recency']]

# Instantiate
scaler = StandardScaler()

# fit_transform
rfm_df_scaled = scaler.fit_transform(merged_rfm)
rfm_df_scaled.shape

In [ ]:
rfm_df_scaled = pd.DataFrame(rfm_df_scaled)
rfm_df_scaled.columns = ['Amount', 'Frequency', 'Recency']
rfm_df_scaled.head()

# Building the Model


### K-Means Clustering

##### K-means clustering is one of the simplest and popular unsupervised machine learning algorithms.

**The algorithm works as follows:**

1-First we initialize k points, called means, randomly.

2-We categorize each item to its closest mean and we update the mean’s coordinates, which are the averages of the items categorized in that mean so far.

3-We repeat the process for a given number of iterations and at the end, we have our clusters.

In [ ]:
# k-means with some arbitrary k

kmeans = KMeans(n_clusters=4, max_iter=50)
kmeans.fit(rfm_df_scaled)

In [ ]:
kmeans.labels_


### Finding the Optimal Number of Clusters


#### Elbow Curve to get the right number of Clusters

###### A fundamental step for any unsupervised algorithm is to determine the optimal number of clusters into which the data may be clustered. The Elbow Method is one of the most popular methods to determine this optimal value of k.

In [ ]:

ssd = []
range_n_clusters = [2, 3, 4, 5, 6, 7, 8]

for num_clusters in range_n_clusters:
    kmeans = KMeans(n_clusters=num_clusters, max_iter=50)
    kmeans.fit(rfm_df_scaled)

    ssd.append(kmeans.inertia_)
    print("For n_clusters={0}, the Elbow score is {1}".format(num_clusters, kmeans.inertia_))

fig = px.line(x=range_n_clusters, y=ssd,
              title="Elbow Curve for K-Means Clustering",
              labels={'x': 'Number of Clusters', 'y': 'Sum of Squared Distances (SSD)'})

fig.update_layout(
    xaxis=dict(title_font=dict(size=14)),
    yaxis=dict(title_font=dict(size=14)),
    showlegend=False,
    width=800,
    height=600
)

fig.show()


In [ ]:
ssd

## Silhouette Analysis

####  silhouette score=p−q/max(p,q)

1-**p**
  
  **is the mean distance to the points in the nearest cluster that the data point is not a part of**

2-**q
  is the mean intra-cluster distance to all the points in its own cluster.**


3-**The value of the silhouette score range lies between -1 to 1.**

**A score closer to 1 indicates that the data point is very similar to other data points in the cluster,**

**A score closer to -1 indicates that the data point is not similar to the data points in its cluster.**

#### Silhouette analysis

helps you assess the quality of clustering and find the number of clusters that best fits your data. Let's break down the code and understand its purpose:

**Purpose of Silhouette Analysis:**

Silhouette analysis helps you assess the quality of clustering for different numbers of clusters.


The silhouette score ranges from -1 to 1, with higher values indicating better cluster separation and cohesion.


By examining silhouette scores for various cluster numbers, you can determine the optimal number of clusters that best fit your data.


In summary, this code performs silhouette analysis for a range of cluster numbers using K-Means clustering on the standardized RFM data. It helps you select the number of clusters that provides the best clustering quality for your data.

In [ ]:
# Silhouette analysis
range_n_clusters = [2, 3, 4, 5, 6, 7, 8]

for num_clusters in range_n_clusters:

    # intialise kmeans
    kmeans = KMeans(n_clusters=num_clusters, max_iter=50)
    kmeans.fit(rfm_df_scaled)

    cluster_labels = kmeans.labels_

    # silhouette score
    silhouette_avg = silhouette_score(rfm_df_scaled, cluster_labels)
    print("For n_clusters={0}, the silhouette score is {1}".format(num_clusters, silhouette_avg))


#### In the context of clustering, when you have multiple options for the number of clusters, you typically aim to choose the number of clusters that maximizes the silhouette score. In your provided output, you have calculated silhouette scores for different numbers of clusters, and you want to select the best number.

#### Here's a simplified explanation for why "3" was chosen from the provided silhouette scores:

**Silhouette Score Meaning:** The silhouette score measures how similar each data point in a cluster is to the data points in the same cluster compared to other clusters. The score ranges from -1 to 1:**

**A higher score indicates better cluster separation and cohesion.**

**A score close to 0 suggests overlapping or ambiguous clusters.**

**A negative score means that data points might be assigned to the wrong clusters.**

Choosing the Best Number of Clusters: To determine the best number of clusters, you typically look for the number that gives the highest silhouette score.

In your provided silhouette scores:

For 2 clusters, the silhouette score is 0.5416.


For 3 clusters, the silhouette score is 0.5085.

For 4 clusters, the silhouette score is 0.4816.

For 5 clusters, the silhouette score is 0.4646.

For 6 clusters, the silhouette score is 0.4176.

For 7 clusters, the silhouette score is 0.4148.

For 8 clusters, the silhouette score is 0.4077.

==> You chose "3" as the number of clusters because it has the second-highest silhouette score (0.5085),

==> and it indicates reasonably good cluster separation and cohesion.

==> While the silhouette score for 2 clusters is slightly higher, the difference is not substantial,

==> and having 3 clusters may provide a more detailed and meaningful separation of your data.

==> Keep in mind that the choice of the number of clusters may also depend on the specific problem and the insights you want to gain from the clustering analysis.

It's often a balance between finding a simple model (few clusters) and capturing meaningful patterns (more clusters).

## Silhouette Analysis Vs Elbow:


#### Imagine you have a group of friends, and you want to split them into different teams for a game. Silhouette analysis is like a way to see how well your friends fit into their teams.

**Purpose:**

Silhouette analysis helps you figure out how good the teams are. It looks at whether your friends are happy with their teams or if they should switch to another team to be even happier.**


**Measurement:**

It gives each friend a score from -1 to 1. If the score is close to 1, it means they're really happy in their team. If it's close to 0, it means they might be confused because their team is similar to other teams. And if it's negative, it means they're not in the right team.


**Goal:**

You want to find the number of teams that makes everyone as happy as possible, with the highest average (average means adding up all the scores and dividing by the number of friends).



Elbow Method:

Now, let's say you want to know how many teams to make for a different game with your friends. The elbow method helps you decide how many teams to create.

**Purpose:**

The elbow method helps you find the right number of teams for your game.

You don't want too few teams, and you don't want too many. You want to strike a balance.


**Measurement:**

Imagine you have a graph that shows how good the teams are. On the graph, you have the number of teams on one side and how well they're organized on the other side. The graph looks like a bent arm with an elbow.



**Goal:**

You want to find the point where the graph starts to bend like an elbow. That point tells you the right number of teams. If you have too few or too many teams, it won't be a clear bend, and that's not what you want.


**Summary:**

**Silhouette analysis** checks **how happy your friends** are in their teams using scores from -1 to 1.

You want the highest average score for the best number of teams.


The elbow method looks at a graph and finds the point where it bends like an elbow. **That point tells you how many teams** to create for your game.

Both methods help you figure out the best way to group things, whether it's friends or data, by considering how happy or organized they are.3

## When choosing the number of clusters for a clustering problem, it's important to consider both the silhouette score and the elbow method results, as they provide complementary insights. Here's how to make the decision based on your provided results:

### Silhouette Score:

The silhouette score measures the quality of the clusters.

Higher silhouette scores indicate better separation and cohesion of clusters.

In your case, the highest silhouette score is for 2 clusters (0.5416), but the second-highest score is for 3 clusters (0.5085), which is still quite good.


### Elbow Method:

The elbow method looks at the reduction in error (WCSS) as you increase the number of clusters.

The point where adding more clusters doesn't significantly reduce the error is often considered the "elbow" and a reasonable choice for the number of clusters.


### Decision:

**Silhouette Score:** The silhouette score suggests that 2 clusters or 3 clusters are both reasonable choices. Both options have good silhouette scores, with 2 clusters having a slightly higher score.

**Elbow Method:** The elbow method suggests that the "elbow" point occurs at 3 clusters. This is where the reduction in error starts to slow down significantly.

### Conclusion:

Based on the silhouette score, you could choose either 2 or 3 clusters, and both are valid choices.

However, considering the results from both methods and the fact that the elbow method indicates a clear point at 3 clusters, it may be a good idea to choose 3 clusters as the number of clusters for your clustering analysis.

Ultimately, the choice between 2 and 3 clusters depends on your specific problem, the insights you want to gain, and your preference for having a more detailed or simplified grouping of your data.

You can start with 3 clusters and, if needed, explore the results further to see if they make sense for your particular application.


In [ ]:
# Final model with k=3
kmeans = KMeans(n_clusters=3, max_iter=50)
kmeans.fit(rfm_df_scaled)

In [ ]:
 kmeans.labels_


In [ ]:
# assign the label
merged_rfm['Cluster_Id'] = kmeans.labels_
merged_rfm.head()

In [ ]:

fig = px.box(merged_rfm, x='Cluster_Id', y='Amount',
             title="Cluster Id vs Amount Box Plot",
             labels={'Cluster_Id': 'Cluster ID', 'Amount': 'Amount'},
             color='Cluster_Id')

fig.update_layout(
    xaxis=dict(title="Cluster ID", title_font=dict(size=14)),
    yaxis=dict(title="Amount", title_font=dict(size=14)),
    showlegend=False,
    width=800,
    height=600
)

fig.show()


In [ ]:

fig = px.scatter(merged_rfm, x='Cluster_Id', y='Amount', color='Frequency',
                 title='Cluster ID vs Amount (Color Encoded by Frequency)',
                 labels={'Cluster_Id': 'Cluster ID', 'Amount': 'Amount', 'Frequency': 'Frequency'})

fig.update_layout(
    xaxis=dict(title="Cluster ID", title_font=dict(size=14)),
    yaxis=dict(title="Amount", title_font=dict(size=14)),
    showlegend=True,
    width=800,
    height=600
)

fig.show()


In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

# Create a custom color palette with Red, Green, and Blue
custom_palette = sns.color_palette(["#FF0000", "#00FF00", "#0000FF"])

# Create a scatter plot matrix with separate plots for each cluster, custom palette, and a larger size
sns.set(style="ticks")
sns.pairplot(merged_rfm, hue='Cluster_Id', vars=['Amount', 'Frequency'], palette=custom_palette, height=4, aspect=1.5)
plt.suptitle('Cluster ID vs Amount & Frequency', y=1.02)
plt.show()


In [ ]:

fig = px.box(merged_rfm, x='Cluster_Id', y='Frequency',
             title="Cluster Id vs Frequency Box Plot",
             labels={'Cluster_Id': 'Cluster ID', 'Frequency': 'Frequency'})

fig.update_layout(
    xaxis=dict(title="Cluster ID", title_font=dict(size=14)),
    yaxis=dict(title="Frequency", title_font=dict(size=14)),
    showlegend=False,
    width=800,
    height=600
)

fig.show()


# Box plot to visualize Cluster Id vs Recency



In [ ]:

fig = px.box(merged_rfm, x='Cluster_Id', y='Recency',
             title="Cluster Id vs Recency Box Plot",
             labels={'Cluster_Id': 'Cluster ID', 'Recency': 'Recency'})

fig.update_layout(
    xaxis=dict(title="Cluster ID", title_font=dict(size=14)),
    yaxis=dict(title="Recency", title_font=dict(size=14)),
    showlegend=False,
    width=800,
    height=600
)

fig.show()


In [ ]:
# Selecting the two features for clustering
data_for_clustering = merged_rfm[['Amount', 'Frequency']]

# Specify the number of clusters (e.g., 3)
n_clusters = 3

# Apply K-Means clustering
kmeans = KMeans(n_clusters=n_clusters)
merged_rfm['Cluster_2D'] = kmeans.fit_predict(data_for_clustering)

# Create a scatter plot with Plotly
fig = px.scatter(merged_rfm, x='Amount', y='Frequency', color='Cluster_2D',
                 title='Clustering by Amount and Frequency',
                 labels={'Amount': 'Amount', 'Frequency': 'Frequency', 'Cluster_2D': 'Cluster'})

fig.update_layout(
    xaxis=dict(title="Amount", title_font=dict(size=14)),
    yaxis=dict(title="Frequency", title_font=dict(size=14)),
    width=800,
    height=600
)

fig.show()


In [ ]:
# Selecting the three features for clustering
data_for_clustering = merged_rfm[['Amount', 'Frequency', 'Recency']]

# Apply K-Means clustering
kmeans = KMeans(n_clusters=n_clusters)
merged_rfm['Cluster_3D'] = kmeans.fit_predict(data_for_clustering)

# Visualize the clusters (scatter plot matrix)
sns.set(style="ticks")
sns.pairplot(merged_rfm, hue='Cluster_3D', vars=['Amount', 'Frequency', 'Recency'], palette='Set1')
plt.suptitle('Clustering by Amount, Frequency, and Recency', y=1.02)
plt.show()


In [ ]:
import plotly.express as px

# Apply K-Means clustering
kmeans = KMeans(n_clusters=n_clusters)
merged_rfm['Cluster_3D'] = kmeans.fit_predict(data_for_clustering)

# Create a 3D scatter plot matrix with Plotly
fig = px.scatter_3d(merged_rfm, x='Amount', y='Frequency', z='Recency', color='Cluster_3D',
                     labels={'Amount': 'Amount', 'Frequency': 'Frequency', 'Recency': 'Recency', 'Cluster_3D': 'Cluster'})

fig.update_layout(
    scene=dict(
        xaxis_title='Amount',
        yaxis_title='Frequency',
        zaxis_title='Recency',
    ),
    title='Clustering by Amount, Frequency, and Recency',
    width=800,
    height=600
)

fig.show()


## Hierarchical Clustering
Hierarchical clustering involves creating clusters that have a predetermined ordering from top to bottom. For example, all files and folders on the hard disk are organized in a hierarchy. There are two types of hierarchical clustering,

Divisive

Agglomerative.

**Single Linkage:**

In single linkage hierarchical clustering, the distance between two clusters is defined as the shortest distance between two points in each cluster. For example, the distance between clusters “r” and “s” to the left is equal to the length of the arrow between their two closest points.

In [ ]:
# Single linkage:

mergings = linkage(rfm_df_scaled, method="single", metric='euclidean')
dendrogram(mergings)
plt.show()

**Complete Linkage**

In complete linkage hierarchical clustering, the distance between two clusters is defined as the longest distance between two points in each cluster. For example, the distance between clusters “r” and “s” to the left is equal to the length of the arrow between their two furthest points.

In [ ]:
# Complete linkage

mergings = linkage(rfm_df_scaled, method="complete", metric='euclidean')
dendrogram(mergings)
plt.show()

**Average Linkage:**

In average linkage hierarchical clustering, the distance between two clusters is defined as the average distance between each point in one cluster to every point in the other cluster. For example, the distance between clusters “r” and “s” to the left is equal to the average length each arrow between connecting the points of one cluster to the other.

In [ ]:
# Average linkage

mergings = linkage(rfm_df_scaled, method="average", metric='euclidean')
dendrogram(mergings)
plt.show()

#### Cutting the Dendrogram based on K


In [ ]:
# 3 clusters
cluster_labels = cut_tree(mergings, n_clusters=3).reshape(-1, )
cluster_labels

In [ ]:
# Assign cluster labels

merged_rfm['Cluster_Labels'] = cluster_labels
merged_rfm.head()

In [ ]:

fig = px.box(merged_rfm, x='Cluster_Labels', y='Amount',
             title="Cluster Labels vs Amount Box Plot",
             labels={'Cluster_Labels': 'Cluster Labels', 'Amount': 'Amount'})

fig.update_layout(
    xaxis=dict(title="Cluster Labels", title_font=dict(size=14)),
    yaxis=dict(title="Amount", title_font=dict(size=14)),
    showlegend=False,
    width=800,
    height=600
)

fig.show()


In [ ]:

fig = px.box(merged_rfm, x='Cluster_Labels', y='Frequency',
             title="Cluster Labels vs Frequency Box Plot",
             labels={'Cluster_Labels': 'Cluster Labels', 'Frequency': 'Frequency'})

fig.update_layout(
    xaxis=dict(title="Cluster Labels", title_font=dict(size=14)),
    yaxis=dict(title="Frequency", title_font=dict(size=14)),
    showlegend=False,
    width=800,
    height=600
)

fig.show()


In [ ]:

fig = px.box(merged_rfm, x='Cluster_Labels', y='Recency',
             title="Cluster Labels vs Recency Box Plot",
             labels={'Cluster_Labels': 'Cluster Labels', 'Recency': 'Recency'})

fig.update_layout(
    xaxis=dict(title="Cluster Labels", title_font=dict(size=14)),
    yaxis=dict(title="Recency", title_font=dict(size=14)),
    showlegend=False,
    width=800,
    height=600
)

fig.show()


In [ ]:
# Pairwise scatter plots for clustering (2D)
plt.figure(figsize=(15, 8))  # Adjust the figure size here

# Amount vs. Frequency
sns.scatterplot(x='Amount', y='Frequency', hue='Cluster_Labels', data=merged_rfm, palette='Set1')
plt.title('Amount vs. Frequency');


In [ ]:
plt.figure(figsize=(15, 8))  # Adjust the figure size here

sns.scatterplot(x='Amount', y='Recency', hue='Cluster_Labels', data=merged_rfm, palette='Set1')
plt.title('Amount vs. Recency');




In [ ]:
# Frequency vs. Recency
plt.figure(figsize=(15, 8))  # Adjust the figure size here
sns.scatterplot(x='Frequency', y='Recency', hue='Cluster_Labels', data=merged_rfm, palette='Set1')
plt.title('Frequency vs. Recency')

plt.tight_layout()
plt.show()

In [ ]:

# Apply K-Means clustering
kmeans = KMeans(n_clusters=n_clusters)
merged_rfm['Cluster_3D'] = kmeans.fit_predict(data_for_clustering)

# Create a 3D scatter plot matrix with Plotly
fig = px.scatter_3d(merged_rfm, x='Amount', y='Frequency', z='Recency', color='Cluster_3D',
                     labels={'Amount': 'Amount', 'Frequency': 'Frequency', 'Recency': 'Recency', 'Cluster_3D': 'Cluster'})

fig.update_layout(
    scene=dict(
        xaxis_title='Amount',
        yaxis_title='Frequency',
        zaxis_title='Recency',
    ),
    title='Clustering by Amount, Frequency, and Recency',
    width=800,
    height=600
)

fig.show()


### Our Analytical results

Inference:

### K-Means Clustering with 3 Cluster Ids

**Customers with Cluster Id 1 are the customers with high amount of transactions as compared to other customers.**

**Customers with Cluster Id 1 are frequent buyers.**

**Customers with Cluster Id 2 are not recent buyers and hence least of importance from business point of view.**

**Hierarchical Clustering with 3 Cluster Labels**

**Customers with Cluster_Labels 2 are the customers with high amount of transactions as compared to other customers.**

**Customers with Cluster_Labels 2 are frequent buyers.**

**Customers with Cluster_Labels 0 are not recent buyers and hence least of importance from business point of view.**